In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('vaccine_preprocessed.csv',sep='|')

In [3]:
df.head(1)

,categories,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
0,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,0.0,"['澎湖5至11歲兒童疫苗接種種率已逾7成', '澎湖縣政府今天公布新增182例確診案例',...","[('澎湖', 5), ('疫苗', 5), ('接種', 4), ('確診', 3), (...","['澎湖', 'COVID-19', '（', '2019', '冠狀', '病毒', '疾...","['澎湖', '冠狀', '病毒', '疾病', '確診數', '澎湖縣', '政府', '...","[NerToken(word='澎湖', ner='GPE', idx=(0, 2)), N...","[('澎湖', 'Nc'), ('COVID-19', 'FW'), ('（', 'PARE...",https://news.google.com/articles/CBMiMmh0dHBzO...


In [4]:
df.shape

(796, 11)

In [5]:
df.groupby('categories').count()

,titles,contents,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,urls
categories,,,,,,,,,,
AZ,200,200,200,200,200,200,200,200,200,200
BNT,200,200,200,200,200,200,200,200,200,200
莫德納,200,200,200,200,200,200,200,200,200,200
高端,196,196,196,196,196,196,196,196,196,196


In [6]:
vaccine_categories= ['AZ', 'BNT', '莫德納', '高端']

# Filter condition: two words and specified POS
# 過濾條件:兩個字以上 特定的詞性
allowedPOS=['Na','Nb','Nc']

In [7]:
# All-in-one function
def get_top_words():
    top_cate_words={} # final result
    counter_all = Counter() # counter for category '全部'
    for category in vaccine_categories:

        df_group = df[df.categories == category]

        # concatenate all filtered words in the same category
        words_group = []
        for row in df_group.token_pos:

            # filter words for each news
            filtered_words =[]
            for (word, pos) in eval(row):
                if (len(word) >= 2) & (pos in allowedPOS):
                    filtered_words.append(word)

            # concatenate filtered words  
            words_group += filtered_words

        # now we can count word frequency
        counter = Counter( words_group )

        # counter 
        counter_all += counter
        topwords = counter.most_common(200)

        # store topwords
        top_cate_words[category]= topwords

    # Process category '全部'
    top_cate_words['全部'] = counter_all.most_common(200)
    
    # To conveniently save data using pandas, we should convert dict to list.
    return list(top_cate_words.items())

In [8]:
%%time
result = get_top_words()

CPU times: total: 1.48 s
Wall time: 1.48 s


In [9]:
result[0:2]

[('AZ',
  [('疫苗', 2976),
   ('中心', 305),
   ('莫德納', 282),
   ('民眾', 280),
   ('疫情', 255),
   ('台灣', 237),
   ('病毒', 220),
   ('醫院', 166),
   ('陳時中', 165),
   ('指揮', 164),
   ('保護力', 162),
   ('血栓', 155),
   ('英國', 155),
   ('抗體', 155),
   ('疾病', 147),
   ('日本', 137),
   ('副作用', 137),
   ('人員', 129),
   ('國家', 124),
   ('中央', 115),
   ('阿斯特捷利康', 115),
   ('長者', 114),
   ('症狀', 113),
   ('反應', 113),
   ('時間', 109),
   ('政府', 108),
   ('莊人祥', 108),
   ('對象', 108),
   ('個案', 106),
   ('國內', 103),
   ('美國', 99),
   ('醫師', 94),
   ('輝瑞', 90),
   ('新冠', 87),
   ('狀況', 83),
   ('數據', 78),
   ('試驗', 76),
   ('肺炎', 75),
   ('冠狀', 73),
   ('資料', 73),
   ('專家', 67),
   ('風險', 67),
   ('衛生局', 66),
   ('結果', 64),
   ('全球', 62),
   ('意願', 62),
   ('衛生', 61),
   ('公司', 61),
   ('間隔', 61),
   ('事件', 61),
   ('身體', 61),
   ('變異株', 60),
   ('記者會', 58),
   ('國際', 58),
   ('大學', 57),
   ('廠牌', 57),
   ('效果', 54),
   ('血小板', 52),
   ('問題', 52),
   ('報告', 51),
   ('世界', 51),
   ('診所', 50),
   ('效力', 50),
   

In [10]:
top_group_words = get_top_words()

In [11]:
df_top_group_words = pd.DataFrame(top_group_words, columns = ['categories','top_keys'])

In [12]:
df_top_group_words

,categories,top_keys
0,AZ,"[(疫苗, 2976), (中心, 305), (莫德納, 282), (民眾, 280),..."
1,BNT,"[(疫苗, 2320), (兒童, 491), (疫情, 303), (中心, 274), ..."
2,莫德納,"[(疫苗, 2506), (莫德納, 1005), (兒童, 481), (疫情, 315)..."
3,高端,"[(疫苗, 2516), (莫德納, 651), (試驗, 258), (疫情, 256),..."
4,全部,"[(疫苗, 10318), (莫德納, 2194), (兒童, 1228), (疫情, 11..."


In [13]:
df_top_group_words.to_csv('vaccine_topkey_with_category_via_token_pos.csv', index=False)